In [ ]:
import pandas as pd
import numpy as np

### Import the data

The dataset shows Airbnb activity in Seattle, has been provided by Airbnb and can be downloaded on Kaggle: <br> 
https://www.kaggle.com/datasets/airbnb/seattle/data <\br>

The dataset contains three files with the following content:
- calender.csv
- listings.csv
- reviews.csv